<a href="https://colab.research.google.com/github/Vercaca/ELMo-With-Keras/blob/master/ELMo_with_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf

In [0]:
tf.__version__

'1.15.0'

In [0]:
import keras
keras.__version__

'2.2.5'

In [0]:
import tensorflow_hub as hub
from keras import backend as K
import keras.layers as layers
from keras.models import Model, load_model
from keras.engine import Layer

import numpy as np

## Keras Version

In [0]:
class ElmoEmbeddingLayer(Layer):
  def __init__(self, **kwargs):
    print('[__init__]')
    super().__init__(**kwargs)
    self.dimensions = 1024
    self.trainable = True
  
  def build(self, input_shape):
    print('[Build] input_shape={}'.format(input_shape))
    self.elmo = hub.Module("https://tfhub.dev/google/elmo/3", 
                           trainable=self.trainable,
                           name="{}_module".format(self.name))
    self.trainable_weights += tf.trainable_variables(
        scope="^{}_module/.*".format(self.name))
    
    super().build(input_shape)
    
  def call(self, x, mask=None):
    print('[call] x.shape = {}'.format(x.shape))
    embeddings = self.elmo(K.squeeze(K.cast(x, tf.string), axis=1),
                  signature="default",
                  as_dict=True)["default"]
    return embeddings
  
  def compute_mask(self, inputs, mask=None):
    print('[compute_mask] inputs type = {}'.format(inputs.shape))
    return backend.not_equal(inputs, '--PAD--')

  def compute_output_shape(self, input_shape):
    print('[compute_output_shape] input_shape = {}'.format(input_shape))
    return (input_shape[0], self.dimensions)
  

In [0]:
def build_model():
  input_text = layers.Input(shape=(1, ), dtype="string")
  embedding = ElmoEmbeddingLayer()(input_text)
  dense = layers.Dense(256, activation='relu')(embedding)
  pred = layers.Dense(1, activation='sigmoid')(dense)

  model = Model(inputs=[input_text], outputs=pred)
  model.compile(loss='binary_crossentropy', 
                optimizer='adam', metrics=['accuracy'])
  model.summary()
  return model

In [0]:
text = [["the cat is on the mat", 1],
         ["dogs are in the fog", 0]]
import pandas as pd
data = pd.DataFrame(text, columns=['text', 'polarity'])

In [0]:
data

,text,polarity
0,the cat is on the mat,1
1,dogs are in the fog,0


In [0]:
train_df = data.copy()

In [0]:
train_text = train_df['text'].tolist()
train_text = [' '.join(t.split()[0:150]) for t in train_text]
train_text = np.array(train_text, dtype=np.dtype('U25'))[:, np.newaxis]
train_label = train_df['polarity'].tolist()


test_text = train_text.copy()
test_label = train_label[:]

train_text.shape, train_label

((2, 1), [1, 0])

In [0]:
train_text[0][0]

'the cat is on the mat'

In [0]:
# Build and fit
model = build_model()
model.fit(train_text, 
          train_label,
          validation_data=(test_text, test_label),
          epochs=1,
          batch_size=32)

[__init__]
[Build] input_shape=(None, 1)
[call] x.shape = (?, 1)
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


[compute_mask] inputs type = (?, 1)
[compute_output_shape] input_shape = (None, 1)
Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_24 (InputLayer)        (None, 1)                 0         
_________________________________________________________________
elmo_embedding_layer_18 (Elm (None, 1024)              4         
_________________________________________________________________
dense_26 (Dense)             (None, 256)               262400    
_________________________________________________________________
dense_27 (Dense)             (None, 1)                 257       
Total params: 262,661
Trainable params: 262,661
Non-trainable params: 0
_________________________________________________________________
Train on 2 samples, validate on 2 samples
Epoch 1/1



2/2 [==============================] - 26s 13s/step - loss: 0.6749 - acc: 1.0000 - val_loss: 0.3886 - val_acc: 1.0000


## Tensorflow version

In [0]:
x = keras.Input(shape=(1,), dtype="string")
# x = tf.placeholder(dtype=np.dtype('U25'), shape=(1,1))
K.cast(x, tf.string)

<tf.Tensor 'input_23:0' shape=(?, 1) dtype=string>

In [0]:
with tf.Session() as sess:
  print(sess.run(K.squeeze(K.cast(x, tf.string), axis=1), feed_dict={x: train_text}))

[b'the cat is on the mat' b'dogs are in the fog']


In [0]:
np.array([['you']], dtype=np.dtype())

array([['you']], dtype='<U25')